In [8]:
import robotic as ry
import numpy as np
import time

In [9]:
ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
#ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 0., 'physx/motorKd': 0.})
ry.params_add({'botsim/engine': 'physx'}) 
ry.params_add({'physx/multibody': True})
ry.params_print()

botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody,
bot/useGripper,
bot/blockRealRobot!,
botsim/tau: 0.01,
botsim/hyperSpeed: 1,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBody,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
sim/displayVisualsOnly,
bot/useOptitrack!,
bot/useAudio!,
botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody

In [10]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

In [11]:
bot = ry.BotOp(C, False)

-- kin_physx.cpp:PhysXInterface:779(0) starting PhysX engine ... (multiBody=1)
-- kin_physx.cpp:addGround:237(0) ... done starting PhysX engine
-- kin_physx.cpp:addGround:238(0) creating Configuration within PhysX ...
-- kin_physx.cpp:addLink:258(0) adding link 'world' as static with 1 shapes ( table)
-- kin_physx.cpp:addMultiBody:469(0) adding multibody with base 'l_panda_base' with the following links ...
-- kin_physx.cpp:addMultiBody:509(0) adding multibody link 'l_panda_base' as kinematic with 1 shapes ( l_panda_link0_0) and joint rigid and mass 2.9873
-- kin_physx.cpp:addMultiBody:509(0) adding multibody link 'l_panda_joint1' as dynamic with 1 shapes ( l_panda_link1_0) and joint hingeZ and mass 2.9646
-- kin_physx.cpp:addMultiBody:509(0) adding multibody link 'l_panda_joint2' as dynamic with 1 shapes ( l_panda_link2_0) and joint hingeZ and mass 2.99224
-- kin_physx.cpp:addMultiBody:509(0) adding multibody link 'l_panda_joint3' as dynamic with 1 shapes ( l_panda_link3_0) and joint 

In [6]:
del bot
del C

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


In [ ]:
qHome = bot.get_qHome()
q0 = qHome.copy()
q1 = q0.copy()
q1[1] = q1[1] + .2
print(q0, q1)

[ 0.  -0.5  0.  -2.   0.   2.  -0.5] [ 0.  -0.3  0.  -2.   0.   2.  -0.5]


In [13]:
bot.moveTo(q1)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [14]:
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q0)
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q1)
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q0)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

timeToEnd: -10.574281057932701
timeToEnd: 1.0957189420649769
timeToEnd: 2.1914378841299538


In [15]:
#this reference frame only appears in your workspace C - not the simulation!
target = C.addFrame('target', 'table')
target.setShape(ry.ST.marker, [.1])
target.setRelativePosition([0., .3, .3])
pos = target.getPosition()
cen = pos.copy()
C.view()
def IK(C, pos):
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 0, False) #one phase one time slice problem, with 'delta_t=1', order=0
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) #cost: close to 'current state'
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'target'], ry.OT.eq, [1e1]) #constraint: gripper position
    ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()

    return [komo.getPath()[0], ret]

In [16]:
for t in range(20):
    time.sleep(.1)
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)

    q_target, ret = IK(C, pos)
    print(ret)
    C.setJointState(q_target)
    C.view()

{ time: 0.004539, evals: 6, done: 1, feasible: 1, sos: 0.00698135, f: 0, ineq: 0, eq: 0.00124554 }
{ time: 0.000235, evals: 3, done: 1, feasible: 1, sos: 0.00372664, f: 0, ineq: 0, eq: 0.00247761 }
{ time: 0.000139, evals: 3, done: 1, feasible: 1, sos: 0.00331079, f: 0, ineq: 0, eq: 0.00221915 }
{ time: 0.000156, evals: 4, done: 1, feasible: 1, sos: 0.00298197, f: 0, ineq: 0, eq: 0.000590664 }
{ time: 0.000146, evals: 3, done: 1, feasible: 1, sos: 0.00254628, f: 0, ineq: 0, eq: 0.0016094 }
{ time: 0.000444, evals: 4, done: 1, feasible: 1, sos: 0.0036348, f: 0, ineq: 0, eq: 0.000453148 }
{ time: 0.000462, evals: 3, done: 1, feasible: 1, sos: 0.00355159, f: 0, ineq: 0, eq: 0.00267651 }
{ time: 0.000236, evals: 4, done: 1, feasible: 1, sos: 0.00408879, f: 0, ineq: 0, eq: 0.000340867 }
{ time: 0.000156, evals: 4, done: 1, feasible: 1, sos: 0.00502846, f: 0, ineq: 0, eq: 0.00065324 }
{ time: 0.000134, evals: 3, done: 1, feasible: 1, sos: 0.00497715, f: 0, ineq: 0, eq: 0.00421624 }
{ time: 0

In [17]:
for t in range(100):
    bot.sync(C, .1) #keep the workspace C sync'ed to real/sim, and idle .1 sec
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)

    q_target, ret = IK(C, pos)
    bot.moveTo(q_target, timeCost=5., overwrite=True)

In [18]:
bot.sync(C, .1) #keep the workspace C sync'ed to real/sim, and idle .1 sec
pos = cen + .9 * (pos-cen) + 0.1 * np.random.randn(3)
target.setPosition(pos)

q_target, ret = IK(C, pos)
#bot.moveTo(q_target, timeCost=0.1, overwrite=True)
bot.moveTo(q_target, timeCost=100, overwrite=True)

In [19]:
for t in range(5):
    bot.moveTo(q1)
    bot.wait(C) #same as 'loop sync til keypressed or endOfTime', but also raises user window
    if bot.getKeyPressed()==ord('q'):
        break;

    bot.moveTo(q0)
    bot.wait(C)
    if bot.getKeyPressed()==ord('q'):
        break;

bot.home(C)

In [20]:
bot.gripperMove(ry._left, width=.02)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.gripperMove(ry._left, width=.075)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

-- simulation.cpp:moveGripper:329(1) initiating opening gripper l_panda_joint7 (without releasing obj) width:0.02 speed:0.2
-- simulation.cpp:modConfiguration:868(1) terminating opening gripper l_panda_joint7 at width 0.0198
-- simulation.cpp:moveGripper:329(1) initiating opening gripper l_panda_joint7 (without releasing obj) width:0.075 speed:0.2
-- simulation.cpp:modConfiguration:868(1) terminating opening gripper l_panda_joint7 at width 0.04


In [23]:
del bot
del C

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


In [24]:
ry.params_print()

botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody,
botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody,
bot/useGripper,
bot/blockRealRobot!,
botsim/tau: 0.01,
botsim/hyperSpeed: 1,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBody,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
sim/displayVisualsOnly,
bot/useOptitrack!,
bot/useAudio!,
bot/raiseWindow!,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
KOMO/sparse,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopInners: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/